# Imports

In [27]:
import os
import numpy as np
from scipy.signal import convolve2d
from PIL import Image
import mediapipe as mp  # to get facial landmarks from the face images

# Constants

In [28]:
DATA_DIR = os.path.join(os.getcwd(), "data_rgb")
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TEST_DIR = os.path.join(DATA_DIR, "test")

# Convolutional Neural Network

## Reading Data
Data are face images.  
If image is in grayscale, there is only 1 channel, for RGB image there would be 3 channels.  

This section extracts the facial landmarks from the images and labels.

1. Read images, from each image extract the facial landmarks, remember the image's label (f.e. happy, sad, ...). We now have fiacl landmarks for each image and label names lists.
2. Map the label names list, each label name will now have its own identificator. (f.e. happy = 0, sad = 1, ...). We now have labels list.
3. Convert the labels list into a one-hot encoded vector array.

In [29]:
# initialize MediaPipe Face Mesh model to extract facial landmarks
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)  # static_image_mode=True since we extract from images and not live video

def extract_landmarks(image_array: np.array, is_greyscale: bool) -> np.array:
    
    """
    Extract facial landmarks from the image using MediaPipe.
    :param image_array: 2D numpy array representing the image
    :param is_greyscale: True if the image is greyscale, False image is in RGB
    :return: numpy array containing the normalized facial landmarks
    """
    
    if is_greyscale:  # convert image array to RGB format, since MediaPipe works with RGB
        image_rgb = np.stack((image_array,) * 3, axis=-1)  # convert grayscale to RGB format
    else:
        image_rgb = image_array  # image already is RGB
        
    results = face_mesh.process(image_rgb)

    if not results.multi_face_landmarks:
        return None  # no face detected, return None

    landmarks = results.multi_face_landmarks[0].landmark
    image_height, image_width = image_array.shape[:2]
    
    # normalize the landmarks by the width and height of the image
    landmarks_array = np.array([(lm.x * image_width, lm.y * image_height) for lm in landmarks])

    return landmarks_array

I0000 00:00:1727813164.379600  433392 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M3 Pro
W0000 00:00:1727813164.381338  510229 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [48]:
def get_data(data_dir: str, is_greyscale: bool, image_size=(48, 48)) -> (np.array, np.array):
    
    """
    Reading data_greyscale from data_dir.
    Each image is processed and converted into a numpy array. Then facial landmarks are extrcated from the image array and the facial landmarks are normalized.
    
    Image is expected to be in grayscale (.convert('L')), and its size is expected to be 48x48 (default).
    
    Expected data_greyscale directory tree:
    - data_dir
        - facial_expression_dir1
            - image1
            - image2
            - ...
        - facial_expression_dir2 
        - facial_expression_dir3
        - ...
        - facial_expression_dirN
        
    :param data_dir: directory from which the images are processed
    :param image_size: size of the images, default is 48x48
    :param is_greyscale: True if the images are greyscale, False images are in RGB
    :return: two numpy arrays, first numpy array has stored all the facial landmarks images, and second numpy array has stored all the label names
    """
    
    failed_image_landmarks = {}
    
    image_landmarks = []
    label_names = []
    
    for expression_dirname in sorted(os.listdir(data_dir)):  
        # get every directory, this directory contains images of facial expressions 
        
        expression_dir = os.path.join(data_dir, expression_dirname)
        
        if not os.path.isdir(expression_dir):  # process only directories, skip non-directories - files
            continue
            
        for expression_image in os.listdir(expression_dir):
            # get every image in the directory
            image_path = os.path.join(expression_dir, expression_image)
            
            try:
                
                if is_greyscale:
                    image = Image.open(image_path).convert('L')  # L mode, because images are grayscaled
                else:
                    image = Image.open(image_path).convert("RGB")  # RGB mode, because images are not grayscaled, and therefore they are RGB 

                image = image.resize(image_size)  # resize the image to expected 48x48
                
                # convert image into an array, image is a 2D array
                image_array = np.array(image)
                
                # extract normlized facial landmarks
                landmarks = extract_landmarks(image_array, is_greyscale)
                
                if landmarks is not None:
                    # successfully the facial landmarks were extracted from the image
                    image_landmarks.append(image_array)
                    label_names.append(expression_dirname)  # directory name is already a label name of a facial expression
                else:
                    # failed to get facial landmarks, add to the counter of failed facial landmarks
                    
                    if expression_dirname not in failed_image_landmarks:
                        failed_image_landmarks[expression_dirname] = {
                            "all_images_count": len(os.listdir(expression_dir)),
                            "failed_images": [os.path.split(image_path)[-1]]
                        }
                    else:
                        failed_image_landmarks[expression_dirname]["failed_images"].append(os.path.split(image_path)[-1])
                    
            except Exception as e:
                print(f"Failed to process image: {image_path}")
                print(e)
    
    if failed_image_landmarks:
        print("Failed image landmarks: \n")
        
        all_images_count = 0
        all_failed_images_count = 0
        
        for expression, info_dict in failed_image_landmarks.items():
            
            all_expression_images_count = info_dict["all_images_count"]
            images_failed = info_dict["failed_images"]
            
            n_failed_images = len(images_failed)
            fail_ratio = round(n_failed_images / all_expression_images_count * 100, 2)
            
            print(f"{expression.upper()} - FAILED: {n_failed_images}/{all_expression_images_count} - {fail_ratio}%")
            for image_failed in images_failed:
                print(f"\t - {image_failed}")
            print("")
            
            all_images_count += all_expression_images_count
            all_failed_images_count += n_failed_images
        
        overall_fail_ratio = round(all_failed_images_count / all_images_count * 100, 2)
        print(f"OVERALL - FAILED: {all_failed_images_count}/{all_images_count} - {overall_fail_ratio}%")
            
    # convert images and labels list into numpy arrays adn return them
    return np.array(image_landmarks), np.array(label_names)            

In [49]:
train_image_landmarks, train_label_names = get_data(TRAIN_DIR, is_greyscale=False, image_size=(128, 128))

Failed image landmarks: 

ANGRY - FAILED: 11/1175 - 0.94%
	 - cropped_emotions.158037~angry.png
	 - cropped_emotions.231444~angry.png
	 - b389cdd600862ce3acfecaaafed2f444e6d5f4a49249b80d3b3fe458~angry.jpg
	 - cropped_emotions.505992~angry.png
	 - cropped_emotions.157957~angry.png
	 - cropped_emotions.505802~angry.png
	 - 6d01a9729bb4b4ad81b039ac13652ce6c03c312c28bbff5672943df4~angry.jpg
	 - 0dc59c2bc0d07d830947534205256c306f6982e8ddcde0b39d2d991f~angry.jpg
	 - cropped_emotions.231896~angry.png
	 - cropped_emotions.231606~angry.png
	 - cropped_emotions.506057~angry.png

HAPPY - FAILED: 12/2909 - 0.41%
	 - cropped_emotions.569571.png
	 - cropped_emotions.506066.png
	 - cropped_emotions.505964.png
	 - cropped_emotions.505782.png
	 - cropped_emotions.505768.png
	 - 0c0bbcbcf39bd2fa6adf6c99c6cea4bea652d554f0324dcae6b98d2e.jpg
	 - 0e19f6d0fea30d22759e1243671c8e3dda4c9ea8c857925f8374445d.jpg
	 - cropped_emotions.505759.png
	 - cropped_emotions.506097.png
	 - cropped_emotions.506135.png
	 - cr

In [50]:
test_image_landmarks, test_label_names = get_data(TEST_DIR, is_greyscale=False, image_size=(128, 128))

Failed image landmarks: 

ANGRY - FAILED: 1/131 - 0.76%
	 - cropped_emotions.231456~angry.png

HAPPY - FAILED: 4/728 - 0.55%
	 - 0a53bca982e8866db87a5e28a287f2d611193aa0b6d79afe9027a9ce.jpg
	 - cropped_emotions.505787.png
	 - cropped_emotions.505810.png
	 - 1a08641071f10b7f0760198c6b452594b1c60ce95c948ffcbbff5766.jpg

NEUTRAL - FAILED: 3/790 - 0.38%
	 - cropped_emotions.278146f.png
	 - cropped_emotions.452763f.png
	 - 1a4733af85d6fce3a89e6b9d56743f42a538559c3e2bacf0a11bc885f.jpg

SAD - FAILED: 4/765 - 0.52%
	 - 5f5c6a9881321668025291f83b6849c148e41e9ce3e19187c8d39a66.jpg
	 - cropped_emotions.171835.png
	 - 6fbc4cef2d9c7b6d5fd1edf10ec888e89ff77b27c9d9474965c40553.jpg
	 - 7f07f0d3a27ccba82b024ef149190b12a7df9bb815c2120d07d85336.jpg

OVERALL - FAILED: 12/2414 - 0.5%


In [51]:
print("Train Image Landmarks:")
print(train_image_landmarks.shape)
print(train_image_landmarks, '\n')

print("Test Image Landmarks:")
print(test_image_landmarks.shape)
print(test_image_landmarks, '\n')

print("Train Labels:")
print(train_label_names.shape)
print(train_label_names, '\n')

print("Test Labels:")
print(test_label_names.shape)
print(test_label_names, '\n')

Train Image Landmarks:
(11354, 128, 128, 3)
[[[[108  13  21]
   [109  14  22]
   [107  15  20]
   ...
   [ 75  41  35]
   [ 71  34  30]
   [ 68  28  26]]

  [[121  20  30]
   [117  17  25]
   [113  15  20]
   ...
   [ 75  43  35]
   [ 73  36  32]
   [ 70  32  29]]

  [[132  24  34]
   [126  19  28]
   [119  14  20]
   ...
   [ 79  46  39]
   [ 77  43  39]
   [ 76  38  35]]

  ...

  [[145 104  80]
   [140  99  75]
   [129  89  64]
   ...
   [ 45  27  27]
   [ 45  25  24]
   [ 44  25  21]]

  [[165 120  89]
   [158 113  83]
   [147 102  72]
   ...
   [ 44  26  26]
   [ 45  25  24]
   [ 46  27  23]]

  [[177 128  95]
   [169 120  87]
   [158 113  79]
   ...
   [ 44  26  26]
   [ 45  27  25]
   [ 47  28  24]]]


 [[[  0   0   0]
   [  0   0   0]
   [  0   0   0]
   ...
   [  0   0   0]
   [  0   0   0]
   [  0   0   0]]

  [[  0   0   0]
   [  0   0   0]
   [  0   0   0]
   ...
   [  0   0   0]
   [  0   0   0]
   [  0   0   0]]

  [[ 24   2   1]
   [ 25   2   1]
   [ 24   2   1]
   ...
 

In [52]:
print("Example landmarks of one image:")
print(train_image_landmarks[0].shape)
print(train_image_landmarks[0])

Example landmarks of one image:
(128, 128, 3)
[[[108  13  21]
  [109  14  22]
  [107  15  20]
  ...
  [ 75  41  35]
  [ 71  34  30]
  [ 68  28  26]]

 [[121  20  30]
  [117  17  25]
  [113  15  20]
  ...
  [ 75  43  35]
  [ 73  36  32]
  [ 70  32  29]]

 [[132  24  34]
  [126  19  28]
  [119  14  20]
  ...
  [ 79  46  39]
  [ 77  43  39]
  [ 76  38  35]]

 ...

 [[145 104  80]
  [140  99  75]
  [129  89  64]
  ...
  [ 45  27  27]
  [ 45  25  24]
  [ 44  25  21]]

 [[165 120  89]
  [158 113  83]
  [147 102  72]
  ...
  [ 44  26  26]
  [ 45  25  24]
  [ 46  27  23]]

 [[177 128  95]
  [169 120  87]
  [158 113  79]
  ...
  [ 44  26  26]
  [ 45  27  25]
  [ 47  28  24]]]


### Map the label names into actual labels
Each label name should have its integer identificator.  
From label names list get labels list, where label name has been replaced by its identificator

In [53]:
def map_label_names(label_names: np.array) -> np.array:
    
    """
    Map the label names, each label name will have its own identificator.
    Replace the label names with their unique identifier.
    :param label_names: list of label names
    :return: an array of labels, where now the label names have been replaced by their unique identifier.
    """
    
    mapped_labels = {}
    
    # map the unique label names
    for label, unique_label_name in enumerate(np.unique(label_names)):
        mapped_labels[unique_label_name] = label

    # replace label name by its identificator
    labels = np.array([mapped_labels[label_name] for label_name in label_names])
    
    return labels

In [54]:
train_labels = map_label_names(train_label_names)
test_labels = map_label_names(test_label_names)

print("Train labels:")
print(train_labels.shape)
print(train_labels, '\n')

print("Test labels:")
print(test_labels.shape)
print(test_labels)

Train labels:
(11354,)
[0 0 0 ... 4 4 4] 

Test labels:
(2526,)
[0 0 0 ... 4 4 4]


### Encode the labels into one hot vectors


In [55]:
def one_hot_encode(labels: np.array, num_classes: int):
    
    """
    Encode the labels 1D vector into one-hot vectors encoding.
    One hot encoded vector has all zeros, but only one 1.
    
    :param labels: 1D vector of labels 
        F.e.:
        happy: 0
        sad: 1
        angry: 2
        labels: [0, 0, 1, 1, 2]
        
    :param num_classes: number of unique classes - of unique labels (f.e. 3 - happy, sad, and angry)
    :return: a 2D one hot encoded array.
    
            F.e.:
            labels = [0, 0, 1, 1, 2]
            one_hot = 
                [
                    [1 0 0]
                    [1 0 0]
                    [0 1 0]
                    [0 1 0]
                    [0 0 1]
                ]
            - shape of one_hot: (n_labels, unique_labels)
    """
    
    # an array full of zeros of shape (num_labels, num_classes) 
    one_hot = np.zeros((len(labels), num_classes))
    
    # set the 1 to appropriate labels
    for n_row, label in enumerate(labels):
        one_hot[n_row, label] = 1
    
    return one_hot

In [56]:
num_classes = len(np.unique(test_labels))

train_labels_one_hot = one_hot_encode(train_labels, num_classes)
test_labels_one_hot = one_hot_encode(test_labels, num_classes)

print("Train labels one-hot:")
print(train_labels_one_hot.shape)
print(train_labels_one_hot, '\n')

print("Test labels one-hot:")
print(test_labels_one_hot.shape)
print(test_labels_one_hot, '\n')

Train labels one-hot:
(11354, 5)
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]] 

Test labels one-hot:
(2526, 5)
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]] 



### Final Representation of Data


Training and testing sets:  
- image arrays: train_images and test_images
- one-hot encoded vector arrays: train_labels_one_hot, test_labels_one_hot

In [58]:
print("Train Images (first):")
print(train_image_landmarks.shape)
print(train_image_landmarks[0], '\n')

print("Test Images (first):")
print(test_image_landmarks.shape)
print(test_image_landmarks[0], '\n')

print("Train Labels one-hot:")
print(train_labels_one_hot.shape)
print(train_labels_one_hot, '\n')

print("Test Labels one-hot:")
print(test_labels_one_hot.shape)
print(test_labels_one_hot, '\n')

Train Images (first):
(11354, 128, 128, 3)
[[[108  13  21]
  [109  14  22]
  [107  15  20]
  ...
  [ 75  41  35]
  [ 71  34  30]
  [ 68  28  26]]

 [[121  20  30]
  [117  17  25]
  [113  15  20]
  ...
  [ 75  43  35]
  [ 73  36  32]
  [ 70  32  29]]

 [[132  24  34]
  [126  19  28]
  [119  14  20]
  ...
  [ 79  46  39]
  [ 77  43  39]
  [ 76  38  35]]

 ...

 [[145 104  80]
  [140  99  75]
  [129  89  64]
  ...
  [ 45  27  27]
  [ 45  25  24]
  [ 44  25  21]]

 [[165 120  89]
  [158 113  83]
  [147 102  72]
  ...
  [ 44  26  26]
  [ 45  25  24]
  [ 46  27  23]]

 [[177 128  95]
  [169 120  87]
  [158 113  79]
  ...
  [ 44  26  26]
  [ 45  27  25]
  [ 47  28  24]]] 

Test Images (first):
(2526, 128, 128, 3)
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0

## Helpful Functions
Functions that have nothing to do with CNN.  
Helpful functions to make the program run smoothly.

## Convolutional Layers

### Custom Errors
Made custom errors, which can be raised in the convolutional layer.

In [95]:
class InvalidNumberOfFilters(Exception):
    """ Invalid number of filters specified """
    def __init__(self, message="Invalid number of filters specified by the user"):
        self.message = message
        super().__init__(self.message)


class InvalidFilterSize(Exception):
    """ Invalid filter size specified """
    def __init__(self, message="Invalid filter size specified by the user"):
        self.message = message
        super().__init__(self.message)
        
        
class InvalidNumberOfChannels(Exception):
    """ Invalid number of channels specified """
    def __init__(self, message="Invalid number of channels specified by the user"):
        self.message = message
        super().__init__(self.message)
        
        
class InvalidPadding(Exception):
    """ Invalid padding specified """
    def __init__(self, message="Invalid padding specified by the user"):
        self.message = message
        super().__init__(self.message)
        

class InvalidStride(Exception):
    """ Invalid stride specified """
    def __init__(self, message="Invalid stride specified by the user"):
        self.message = message
        super().__init__(self.message)

In [59]:
class ConvLayer:
    
    """
    """
    
    def __init__(self, num_filters: int, filter_size: int, num_channels: int):
        """
        Function constructor, initialize the filter array.
        
        :param num_filters: number of filters in the convolutional layer
        :param filter_size: size of the filter, num_channels x filter_size x filter_size
        :param num_channels: depth of the filter - kernel
        :raises 
        """
        
        if not isinstance(num_filters, int):
            raise ValueError("Number of filters \"num_filters\" must be an integer!")
        elif num_filters < 1:
            raise InvalidNumberOfFilters(f"Number of filters \"num_filters\" must be at least 1, \"num_filters={num_filters}\" specified instead!")
        
        if not isinstance(filter_size, int):
            raise ValueError("Filter - kernel size \"filter_size\" must be an integer!")
        elif filter_size < 2:
            raise InvalidFilterSize(f"Filter - kernel size \"filter_size\" must be at least 2, \"filter_size={filter_size}\" specified instead!")
            
        if not isinstance(num_channels, int):
            raise ValueError("Number of channels \"num_channels\" must be an integer!")
        elif num_channels < 1:
            raise InvalidNumberOfChannels(f"Number of channels \"num_channels\" must be at least 1, \"num_channels={num_channels}\" was specified instead!")
            
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.num_channels = num_channels
        
        # creates a 4D numpy array
        # filter is of size (c x f x f), where f is the filter_size, and c is the num_channels
        # (c x f x f) because image size is expected to be (c x height x width)
        self.filters = np.random.randn(num_filters, num_channels, filter_size, filter_size) / filter_size ** 2
    
    
    def shift_filter_window(self, image: np.array, stride: int) -> (np.array, int, int):
        
        """
        Shift the filter window to get all the regions in the image.
        These regions are for the convolutional operations with the kernel - filter.
        
        :param image: image from which to get all the regions from.
        :param stride: size of the stride, shift size - how big a step/shift
        :return: a region (where input image and kernel intersect) which is a 2D array, and (i, j) coordinates of the region  
        """
        
        _, height, width = image.shape
        
        # shift the filter window, to process all regions in the input image
        for i in range(0, height - self.filter_size + 1, stride):
            for j in range(0, width - self.filter_size + 1, stride):
                # extract the region of the image where the filter is applied
                region = image[:, i: (i + self.filter_size), j: (j + self.filter_size)]
                yield region, i, j  # what yield does that it returns a value, but this function has to be interated to get next() values
                
                
    def forward(self, image: np.array, stride: int = 1, padding: int = 0) -> np.array:

        """
        Perform forward pass through the convolutional layer.
        Perform convolutional operations with the kernel - filter 
        
        :param image: image to perform convolutional operation on 
        :param padding: padding applied to the image
        :param stride: stride applied to the convolutional operation
        :return: a convoluted image
        """
        
        if not isinstance(stride, int):
            raise ValueError("Stride must be an integer!")
        elif stride < 1:
            raise InvalidStride(f"Stride must be at least 1, \"stride={stride}\" specified instead!")
        
        if not isinstance(padding, int):
            raise ValueError("Padding must be an integer!")
        elif padding < 0:
            raise InvalidPadding(f"Padding must be at least 0, \"padding={padding}\" specified instead!")
        
        # if image is 2D (single channel, number of channels is 1), expand from (height x width) into (1 x height x width) so it has a depth=1
        if image.ndim == 2:
            image = np.expand_dims(image, axis=0)
    
        padded_image = self.pad_image(image, padding)  # apply padding to the image
        _, height, width = image.shape  # get only height and width, image is of size (n_channels x height x width)
        
        # expected output height and width
        output_height = (height + 2 * padding - self.filter_size) // stride + 1
        output_width = (width + 2 * padding - self.filter_size) // stride + 1
        
        output = np.zeros((self.num_filters, output_height, output_width))

        for region, i, j in self.shift_filter_window(padded_image, stride):
            for n_filter in range(self.num_filters):
                # perform convolution on the region using the filter
                output[n_filter, i // stride, j // stride] = self.conv(region, self.filters[n_filter]) 
        
        return output
        
    
    @staticmethod
    def conv(image_region: np.array, kernel: np.array) -> np.array:
        
        """
        Perform convolutional operation on the image region using the filter.
        Convolution operation is sum(image_region * filter)
        
        :param image_region: region to perform convolutional operation on
        :param kernel: filter - kernel to perform convolutional operation with
        :return: convolution result
        """
        
        result = 0
  
        # perform convolution for each channel separately and sum the result
        for channel in range(image_region.shape[0]):  # loop over the channels
            conv_result = convolve2d(image_region[channel], kernel[channel], mode="valid")
            result += conv_result.item()  # extract the scalar value if it is a 1 x 1 array

        return result
        
        
    @staticmethod
    def pad_image(image: np.array, padding: int) -> np.array:
        
        """
        Apply zero-padding to the input image with multiple channels.
        
        :param image: image to apply padding on
        :param padding: padding applied to the image
        :return: padded image
        """
        
        if padding > 0:
            # apply padding to the height and width dimensions (1 and 2), keeping channels dimension intact
            padded_image = np.pad(image, ((0, 0), (padding, padding), (padding, padding)), mode="constant")
        else:
            padded_image = image
            
        return padded_image
        

In [62]:
# Example of using one convolutional layer, using only forward pass

conv_layer1 = ConvLayer(num_filters=3, filter_size=4, num_channels=1)

batch_size = 300
convolved_images = []

for i in range(0, train_image_landmarks.shape[0], batch_size):  # loop over all the number of images (28709)
    
    print(f"{(i // batch_size) + 1}. batch {i}/{train_image_landmarks.shape[0]}")
    
    convolved_batch = []
    batch_images = train_image_landmarks[i: i + batch_size]  # Get the i-th image (shape 48x48)
    
    for image in batch_images:
        convolved_image = conv_layer1.forward(image, padding=0, stride=1)
        convolved_batch.append(convolved_image)
    
    convolved_images.extend(np.array(convolved_batch))

convolved_images = np.array(convolved_images)

1. batch 0/11354
2. batch 300/11354
3. batch 600/11354
4. batch 900/11354
5. batch 1200/11354
6. batch 1500/11354
7. batch 1800/11354
8. batch 2100/11354
9. batch 2400/11354
10. batch 2700/11354
11. batch 3000/11354
12. batch 3300/11354
13. batch 3600/11354
14. batch 3900/11354
15. batch 4200/11354
16. batch 4500/11354
17. batch 4800/11354
18. batch 5100/11354
19. batch 5400/11354
20. batch 5700/11354
21. batch 6000/11354
22. batch 6300/11354
23. batch 6600/11354
24. batch 6900/11354
25. batch 7200/11354
26. batch 7500/11354
27. batch 7800/11354
28. batch 8100/11354
29. batch 8400/11354
30. batch 8700/11354
31. batch 9000/11354
32. batch 9300/11354
33. batch 9600/11354
34. batch 9900/11354
35. batch 10200/11354
36. batch 10500/11354
37. batch 10800/11354
38. batch 11100/11354


In [63]:
convolved_images.shape

(11354, 3, 125, 0)